<a href="https://colab.research.google.com/github/Bommireddypally/Miniprojects/blob/main/task5_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Install unrar if not already installed
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
BASE_DIR = "/content"

# Assuming RAR files are in BASE_DIR. Find all .rar files and extract them.
# You might need to adjust BASE_DIR if your rar files are in a different location.

print(f"Extracting RAR files from: {BASE_DIR}")

rar_files = [f for f in os.listdir(BASE_DIR) if f.endswith('.rar')]

if not rar_files:
    print(f"No .rar files found in {BASE_DIR}.")
else:
    for rar_file in rar_files:
        rar_path = os.path.join(BASE_DIR, rar_file)
        print(f"Extracting {rar_file}...")
        # -o+ overwrites existing files without prompting
        !unrar x -o+ "{rar_path}" "{BASE_DIR}/"
    print("All RAR files extracted.")

# Verify content after extraction (optional, but good for debugging)
print("Content of BASE_DIR after extraction:")
print(os.listdir(BASE_DIR))

Extracting RAR files from: /content
Extracting test_1000.rar...

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/test_1000.rar

Creating    /content/test_1000                                        OK
Extracting  /content/test_1000/1.jpg                                       0%  OK 
Extracting  /content/test_1000/10.jpg                                      0%  OK 
Extracting  /content/test_1000/100.jpg                                     0%  OK 
Extracting  /content/test_1000/1000.jpg                                    0%  OK 
Extracting  /content/test_1000/101.jpg                                     0%  OK 
Extracting  /content/test_1000/102.jpg                                     0%  OK 
Extracting  /content/test_1000/103.jpg                                     0%  OK 
Extracting  /content/test_1000/104.jpg                                     0%  OK 
Extracting 

In [ ]:
# =========================================================
# 1. Imports + GPU mixed precision
# =========================================================

import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    GlobalAveragePooling2D,
    Dense,
    Dropout
)
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras import mixed_precision

# Enable mixed precision (GPU optimization)
mixed_precision.set_global_policy('mixed_float16')


# =========================================================
# 2. Paths & Constants
# =========================================================

BASE_DIR = "/content"

TRAIN_PATH = "/content/Train_3000"
TEST_PATH = "/content/test_1000"

IMAGE_SIZE = (128, 128)
BATCH_SIZE = 64
EPOCHS = 5
RANDOM_STATE = 42


# =========================================================
# 3. Sanity checks
# =========================================================

print(tf.config.list_physical_devices('GPU'))
print(len(os.listdir(TRAIN_PATH)))
print(len(os.listdir(TEST_PATH)))


# =========================================================
# 4. Prepare DataFrame (labels from filenames)
# =========================================================

filenames = os.listdir(TRAIN_PATH)

categories = []
for filename in filenames:
    if filename.startswith("cat"):
        categories.append("cat")
    else:
        categories.append("dog")

df = pd.DataFrame({
    "filename": filenames,
    "category": categories
})

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=df["category"]
)


# =========================================================
# 5. Image generators
# =========================================================

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=TRAIN_PATH,
    x_col="filename",
    y_col="category",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory=TRAIN_PATH,
    x_col="filename",
    y_col="category",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)


# =========================================================
# 6. CNN Model (mixed precision safe)
# =========================================================

model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),

    GlobalAveragePooling2D(),

    Dense(128, activation="relu"),
    Dropout(0.5),

    # Output in float32 for numerical stability
    Dense(1, activation="sigmoid", dtype="float32")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


# =========================================================
# 7. Train model
# =========================================================

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)


# =========================================================
# 8. Load test images (unlabeled)
# =========================================================

test_images = []
image_names = os.listdir(TEST_PATH)

for img_name in image_names:
    img_path = os.path.join(TEST_PATH, img_name)
    img = load_img(img_path, target_size=IMAGE_SIZE)
    img = img_to_array(img) / 255.0
    test_images.append(img)

test_images = np.array(test_images)


# =========================================================
# 9. Predict on test data
# =========================================================

predictions = model.predict(test_images)

labels = ["Dog" if pred > 0.5 else "Cat" for pred in predictions]

results = pd.DataFrame({
    "Image": image_names,
    "Prediction": labels
})

print(results.head())


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
3001
1000
Found 2400 validated image filenames belonging to 2 classes.
Found 601 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,889 (429.25 KB)

 Trainable params: 109,889 (429.25 KB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 39s 694ms/step - accuracy: 0.5272 - loss: 0.6953 - val_accuracy: 0.4992 - val_loss: 0.6925
Epoch 2/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 335ms/step - accuracy: 0.4896 - loss: 0.6940 - val_accuracy: 0.5358 - val_loss: 0.6915
Epoch 3/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 333ms/step - accuracy: 0.5493 - loss: 0.6875 - val_accuracy: 0.5324 - val_loss: 0.6834
Epoch 4/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 330ms/step - accuracy: 0.5685 - loss: 0.6767 - val_accuracy: 0.5923 - val_loss: 0.6558
Epoch 5/5
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 332ms/step - accuracy: 0.6002 - loss: 0.6554 - val_accuracy: 0.5890 - val_loss: 0.6512
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
     Image Prediction
0  447.jpg        Cat
1  277.jpg        Dog
2  712.jpg        Dog
3  829.jpg        Cat
4  869.jpg        Dog
